In [1]:
# use open cv to show new images from AirSim 

import setup_path 
import airsim

# requires Python 3.5.3 :: Anaconda 4.4.0

# pip install opencv-python
import cv2
import time
import math
import sys
import numpy as np



In [2]:
from PIL import Image

def getScreenDepthVis():

    responses = client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.DepthPerspective, True, False)])
    img1d = np.array(responses[0].image_data_float, dtype=np.float)
    img1d = 255/np.maximum(np.ones(img1d.size), img1d)
    img2d = np.reshape(img1d, (responses[0].height, responses[0].width))

    image = np.invert(np.array(Image.fromarray(img2d.astype(np.uint8), mode='L')))

    factor = 10
    maxIntensity = 255.0 # depends on dtype of image data

    # Decrease intensity such that dark pixels become much darker, bright pixels become slightly dark 
    newImage1 = (maxIntensity)*(image/maxIntensity)**factor
    newImage1 = np.array(newImage1,dtype=np.uint8)

    #cv2.imshow("Test", newImage1)
    #cv2.waitKey(0)

    return newImage1

In [3]:
import matplotlib.pyplot as plt

In [8]:
client = airsim.MultirotorClient()
client.confirmConnection()
client.enableApiControl(True)
client.armDisarm(True)
client.takeoffAsync().join()
# client.moveByVelocityAsync(0,0,-5,10)
# client.hoverAsync()
# you must first press "1" in the AirSim view to turn on the depth capture

# get depth image
yaw = 0
pi = 3.14159265483
vx = 0
vy = 0
driving = 0
help = False

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)



In [9]:
from PIL import Image

responses = client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.DepthPerspective, True, False)])
img1d = np.array(responses[0].image_data_float, dtype=np.float)
img1d = 255/np.maximum(np.ones(img1d.size), img1d)
img2d = np.reshape(img1d, (responses[0].height, responses[0].width))
        
image = np.invert(np.array(Image.fromarray(img2d.astype(np.uint8), mode='L')))
        
factor = 10
maxIntensity = 255.0 # depends on dtype of image data
        
        # Decrease intensity such that dark pixels become much darker, bright pixels become slightly dark 
newImage1 = (maxIntensity)*(image/maxIntensity)**factor
newImage1 = np.array(newImage1,dtype=np.uint8)
        
        #cv2.imshow("Test", newImage1)
        #cv2.waitKey(0)
        
#         return newImage1

In [10]:
fly_speed=10
turn_speed=0.5
while True:
    # this will return png width= 256, height= 144
    result = client.simGetImage("0", airsim.ImageType.DepthVis)
    if (result == "\0"):
        if (not help):
            help = True
            print("Please press '1' in the AirSim view to enable the Depth camera view")
    else:    
#         rawImage = np.fromstring(result, np.int8)
#         print("rawImage is",rawImage)
#         png = cv2.imdecode(rawImage, cv2.IMREAD_UNCHANGED)
#         gray = cv2.cvtColor(png, cv2.COLOR_BGR2GRAY)
        gray = getScreenDepthVis()
        # slice the image so we only check what we are headed into (and not what is down on the ground below us).
        top = np.vstack((np.vsplit(gray, 3)[0],np.vsplit(gray, 3)[1]))
        #top = np.vsplit(gray, 3)[0]

        # now look at 4 horizontal bands (far left, left, right, far right) and see which is most open.
        # the depth map uses black for close (0) and white for far away(255), so we invert that
        # to get an estimate of distance.
        bands = np.hsplit(top, [50,100,150,200]);
        mins = [np.min(x) for x in bands]
        max = np.argmax(mins)    
        distance = mins[max]

        # sanity check on what is directly in front of us (slot 2 in our hsplit)
        current = mins[2]
        print ("max distance=", current)
        print ("distance=", current)
#         if (current <20):
#             client.hoverAsync().join()
#             airsim.wait_key("whoops - we are about to crash, so stopping!")
    
        pitch, roll, yaw  = airsim.to_eularian_angles(client.simGetVehiclePose().orientation)

        if (distance > current + 10):
        
            # we have a 90 degree field of view (pi/2), we've sliced that into 5 chunks, each chunk then represents
            # an angular delta of the following pi/10.
            change = 0
            driving = max
            if (max == 0):
                change = -2 * pi / 10
            elif (max == 1):
                change = -pi / 10
            elif (max == 2):
                change = 0 # center strip, go straight
            elif (max == 3):
                change = pi / 10
            else:
                change = 2*pi/10
    
            yaw = (yaw + change)
            vx = turn_speed*math.cos(yaw);
            vy = turn_speed*math.sin(yaw);
            print ("switching angle", math.degrees(yaw), vx, vy, min, distance, current)
    
        if (vx == 0 and vy == 0):
            vx = math.cos(yaw);
            vy = math.sin(yaw);

        #print ("distance=", current)
        client.moveByVelocityZAsync(fly_speed*vx, fly_speed*vy,-25, 1, airsim.DrivetrainType.ForwardOnly, airsim.YawMode(False, 0)).join()

        x = int(driving * 50)
        cv2.rectangle(gray, (x,0), (x+50,50), (0,255,0), 2)
        cv2.imshow("Top", top)

    key = cv2.waitKey(1) & 0xFF;
    if (key == 27 or key == ord('q') or key == ord('x')):
        break;
        


max distance= 79
distance= 79
max distance= 117
distance= 117
max distance= 178
distance= 178
switching angle 18.00000039842971 0.47552825707313856 0.15450850049425455 <built-in function min> 193 178
max distance= 200
distance= 200
switching angle -18.200943610281318 0.47498345375942924 -0.15617528183026966 <built-in function min> 226 200
max distance= 217
distance= 217
max distance= 200
distance= 200
max distance= 193
distance= 193
max distance= 178
distance= 178
switching angle -36.203983529615186 0.403459623958254 -0.29533088533958696 <built-in function min> 200 178
max distance= 193
distance= 193
max distance= 193
distance= 193
max distance= 185
distance= 185
max distance= 170
distance= 170
switching angle -0.20461152953114176 0.4999968117344537 -0.001785568643675678 <built-in function min> 193 170
max distance= 185
distance= 185
max distance= 164
distance= 164
switching angle 35.96511841966823 0.4046873435817574 0.2936463075619045 <built-in function min> 178 164
max distance= 178


max distance= 200
distance= 200
max distance= 200
distance= 200
max distance= 193
distance= 193
max distance= 185
distance= 185
switching angle -141.43813667875196 -0.39096778007283045 -0.31167963511420327 <built-in function min> 200 185
max distance= 151
distance= 151
switching angle -125.67704347734842 -0.2916078947661726 -0.4061586336765979 <built-in function min> 226 151
max distance= 117
distance= 117
switching angle -108.7314571982771 -0.16056649534537443 -0.47351705415169987 <built-in function min> 226 117
max distance= 217
distance= 217
max distance= 178
distance= 178
switching angle -90.78851570094854 -0.00688088038362937 -0.4999526512432414 <built-in function min> 200 178
max distance= 217
distance= 217
max distance= 209
distance= 209
max distance= 209
distance= 209
max distance= 200
distance= 200
max distance= 217
distance= 217
max distance= 217
distance= 217
max distance= 209
distance= 209
max distance= 209
distance= 209
max distance= 200
distance= 200
max distance= 185
dis

max distance= 94
distance= 94
max distance= 30
distance= 30
switching angle -195.2597195291197 -0.4823713449573099 0.13159743752853237 <built-in function min> 72 30
max distance= 133
distance= 133
max distance= 117
distance= 117
switching angle 128.83249016601152 -0.3135228211360342 0.38949125872977214 <built-in function min> 178 117
max distance= 112
distance= 112
switching angle 112.48549438747466 -0.19122476043840456 0.46198819356697285 <built-in function min> 170 112
max distance= 90
distance= 90
switching angle 149.7954047346434 -0.43211722785575485 0.25154463101059743 <built-in function min> 209 90
max distance= 209
distance= 209
max distance= 200
distance= 200
max distance= 193
distance= 193
max distance= 185
distance= 185
switching angle 113.79561686183938 -0.20173765029046659 0.4574952682326686 <built-in function min> 200 185
max distance= 193
distance= 193
max distance= 178
distance= 178
max distance= 164
distance= 164
max distance= 133
distance= 133
switching angle 95.795046

max distance= 226
distance= 226
max distance= 226
distance= 226
switching angle -128.83852525081517 -0.3135638453291068 -0.38945823255186685 <built-in function min> 245 226
max distance= 245
distance= 245
max distance= 245
distance= 245
max distance= 235
distance= 235
max distance= 235
distance= 235
max distance= 235
distance= 235
max distance= 235
distance= 235
max distance= 235
distance= 235
max distance= 235
distance= 235
max distance= 235
distance= 235
max distance= 226
distance= 226
switching angle -146.83852690363864 -0.4185661587222888 -0.27350022079052827 <built-in function min> 245 226
max distance= 235
distance= 235
max distance= 235
distance= 235
max distance= 235
distance= 235
max distance= 235
distance= 235
max distance= 235
distance= 235
max distance= 235
distance= 235
max distance= 235
distance= 235
max distance= 235
distance= 235
max distance= 235
distance= 235
max distance= 235
distance= 235
max distance= 226
distance= 226
switching angle -182.83856730158868 -0.4993865

max distance= 255
distance= 255
max distance= 255
distance= 255
max distance= 255
distance= 255
max distance= 255
distance= 255
max distance= 255
distance= 255
max distance= 255
distance= 255


TimeoutError: Request timed out

In [ ]:
plt.imshow(newImage1,plt.cm.gray)

In [ ]:
response=client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.DepthPlanner, pixels_as_float=True)])[0]

In [ ]:
depth = np.array(response.image_data_float)
depth = depth.reshape(response.height, response.width)
#depth = np.array(depth * 255, dtype=np.uint8)


In [ ]:
 np.array(response.image_data_float)[160:180]

In [ ]:
result = client.simGetImage("0", airsim.ImageType.Segmentation)

rawImage = np.fromstring(result, np.int8)
png = cv2.imdecode(rawImage, cv2.IMREAD_UNCHANGED)
gray = cv2.cvtColor(png, cv2.COLOR_BGR2GRAY)

In [ ]:
plt.imshow(png)

In [ ]:
 np.array(depth * 255, dtype=np.uint8)

In [ ]:
depth = np.array(response.image_data_float)
depth = depth.reshape(response.height, response.width)

In [ ]:
depth [0][160:180]

In [ ]:
plt.imshow(depth,plt.cm.gray)

In [ ]:
img1d = np.fromstring(response.image_data_uint8, dtype=np.uint8)
#img_rgb = img1d.reshape(response.height, response.width, 3) # reshape array to 3 channel image array H X W X 3
cv2.imwrite("./try.png", img1d) # write to png 

In [ ]:
img1d = np.fromstring(response.image_data_uint8, dtype=np.uint8) # get numpy array
img_rgb = img1d.reshape(response.height, response.width, 3) # reshape array to 3 channel image array H X W X 3

In [ ]:
len(img1d)

In [ ]:
result_scene =client.simGetImage("0", airsim.ImageType.Scene)
rawImage = np.fromstring(result_scene, np.int8)

In [ ]:
len(rawImage)

In [ ]:
response=(client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.DepthVis)])[0])

In [ ]:
tmp=client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.DepthVis,False, False)])[0].image_data_uint8

In [ ]:
tmp=client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.DepthVis, pixels_as_float=True)])[0]

In [ ]:
rawImage = np.fromstring(tmp, np.int8)

In [ ]:
rawImage = np.frombuffer(tmp, np.int8)

In [ ]:
tmp

In [ ]:
plt.imshow(gray)

In [ ]:
png = cv2.imdecode(rawImage, cv2.IMREAD_UNCHANGED)
gray = cv2.cvtColor(png, cv2.COLOR_BGR2GRAY)

In [ ]:
color = cv2.cvtColor(depth, cv2.COLOR_GRAY2RNG)

In [ ]:
depth[0]

In [ ]:
depth = np.array(response.image_data_float)
depth = depth.reshape(response.height, response.width)
#depth = np.array(depth * 253, dtype=np.uint8)

In [ ]:
depth

In [ ]:
(depth * 255)[0][160:180]

In [ ]:
plt.imshow(depth,)

In [ ]:
response.image_data_float[160:180]

In [ ]:
depth[0]

In [ ]:
depth[0][160:180]

In [ ]:
png = cv2.imdecode(rawImage, cv2.IMREAD_UNCHANGED)

In [ ]:
response.image_data_uint8

In [ ]:
plt.imshow(depth,plt.cm.gray)

In [ ]:
result_scene =client.simGetImage("0", airsim.ImageType.Scene)
rawImage = np.fromstring(result_scene, np.int8)
png = cv2.imdecode(rawImage, cv2.IMREAD_UNCHANGED)

In [ ]:
result =client.simGetImage("4", airsim.ImageType.DepthPlanner)
rawImage = np.fromstring(result, np.int8)
png = cv2.imdecode(rawImage, cv2.IMREAD_UNCHANGED)
gray = cv2.cvtColor(png, cv2.COLOR_BGR2GRAY)

In [ ]:
rawImage=np.int8(np.ones(np.size(rawImage))*0)+rawImage

In [ ]:
np.unique(png)

In [ ]:
rawImage=np.ones(np.size(rawImage))*0+rawImage
png = cv2.imdecode(rawImage, cv2.IMREAD_UNCHANGED)
##gray = cv2.cvtColor(png, cv2.COLOR_BGR2GRAY)

In [ ]:
response=client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.DepthPlanner,False, False)])[0]
img1d = np.fromstring(response.image_data_uint8, dtype=np.uint8) # get numpy array
img_rgb = img1d.reshape(response.height, response.width, 3) # reshape array to 4 channel image array H X W X 3
cv2.imwrite(os.path.normpath(filename + '.png'), img_rgb) # write to png

In [ ]:
cv2.imwrite("./try2.png",response.image_data_uint8)

In [ ]:
fh = open("./try2.png", "wb")
fh.write(response.image_data_uint8)
fh.close()

In [ ]:
png = cv2.imdecode(img1d, cv2.IMREAD_UNCHANGED)

In [ ]:
np.unique(png)

In [ ]:
# call airsim
responses = client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.Scene, True)])
response = responses[0]
# convert to grayscale
depth = np.array(response.image_data_float, dtype=np.float32)
depth = depth.reshape(response.height, response.width)
depth = np.array(depth * 255, dtype=np.uint8)
# save pic
# cv2.imwrite('./depth.png'.format(name_folder, t), depth)

In [ ]:
cv2.imwrite("./filename.png", depth) 

In [ ]:
rawImage = np.fromstring(tmp, np.int8)

In [ ]:
result_depth = client.simGetImage("0", airsim.ImageType.DepthVis)
rawImage = np.fromstring(result_depth, np.int8)

In [ ]:
np.size(rawImage)

In [ ]:
result_DepthPlanner = client.simGetImage("0", airsim.ImageType.DepthPlanner)

In [ ]:
# result_DepthPlanner = client.simGetImage("0", airsim.ImageType.DepthPlanner)
result_DepthPlanner=client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.DepthPlanner, pixels_as_float=True)])

In [ ]:
rawImage = np.fromstring(result_DepthPlanner, np.int8)

In [ ]:
result_DisparityNormalized = client.simGetImage("0", airsim.ImageType.DisparityNormalized)
rawImage = np.fromstring(result_DisparityNormalized, np.int8)

In [ ]:
result_Segmentation  = client.simGetImage("0", airsim.ImageType.Segmentation)
rawImage = np.fromstring(result_Segmentation, np.int8)

In [ ]:
rawImage = np.fromstring(result_DepthPlanner, np.int8)

In [ ]:
png = cv2.imdecode(rawImage, cv2.IMREAD_GRAYSCALE)
# gray = cv2.cvtColor(png, cv2.COLOR_BGR2GRAY)

In [ ]:
plt.imshow(png)

In [ ]:
# call airsim
responses = client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.DepthVis, True)])
response = responses[0]
# convert to grayscale
depth = np.array(response.image_data_float, dtype=np.float32)
depth = depth.reshape(response.height, response.width)
depth = np.array(depth * 255, dtype=np.uint8)
# save pic
# cv2.imwrite('{}/depth{}.png'.format(self.name_folder, self.t), depth)

In [ ]:
plt.imshow(depth)

In [ ]:
np.size(rawImage)

In [ ]:
np.reshape(rawImage,[2,2389])

In [ ]:
len(rawImage)/2

In [ ]:
plt.imshow(png)

In [ ]:
import numpy as np
import urllib
import cv2

url = 'http://www.pyimagesearch.com/wp-content/uploads/2015/01/google_logo.png'
resp = urllib.urlopen(url)
image = np.asarray(bytearray(resp.read()), dtype="uint8")
image = cv2.imdecode(image, cv2.IMREAD_COLOR)
cv2.imshow('URL2Image',image)

In [ ]:
plt.imshow(png)

In [ ]:
top = np.vsplit(gray, 3)[0]

In [ ]:
np.vsplit(gray, 2)

In [ ]:
bands = np.hsplit(top, [50,100,150,200]);

In [ ]:
        maxes = [np.max(x) for x in bands]
        min = np.argmin(maxes)    
        distance = 255 - maxes[min]

        # sanity check on what is directly in front of us (slot 2 in our hsplit)
        current = 255 - maxes[2]

In [ ]:
plt.imshow(bands[5])

In [ ]:
gray = getScreenDepthVis()
# slice the image so we only check what we are headed into (and not what is down on the ground below us).

top = np.vsplit(gray, 2)[0]

# now look at 4 horizontal bands (far left, left, right, far right) and see which is most open.
# the depth map uses black for far away (0) and white for very close (255), so we invert that
# to get an estimate of distance.
bands = np.hsplit(top, [50,100,150,200]);
maxes = [np.max(x) for x in bands]
min = np.argmin(maxes)    
distance = 255 - maxes[min]

# sanity check on what is directly in front of us (slot 2 in our hsplit)
current = 255 - maxes[2]

In [ ]:
bands

In [ ]:
plt.imshow(gray,plt.cm.gray)

In [5]:
fly_speed=10
turn_speed=0.5
while True:
    # this will return png width= 256, height= 144
    result = client.simGetImage("0", airsim.ImageType.DepthVis)
    if (result == "\0"):
        if (not help):
            help = True
            print("Please press '1' in the AirSim view to enable the Depth camera view")
    else:    
#         rawImage = np.fromstring(result, np.int8)
#         print("rawImage is",rawImage)
#         png = cv2.imdecode(rawImage, cv2.IMREAD_UNCHANGED)
#         gray = cv2.cvtColor(png, cv2.COLOR_BGR2GRAY)
        gray = getScreenDepthVis()
        # slice the image so we only check what we are headed into (and not what is down on the ground below us).
        top = np.vstack((np.vsplit(gray, 3)[0],np.vsplit(gray, 3)[1]))
        #top = np.vsplit(gray, 3)[0]

        # now look at 4 horizontal bands (far left, left, right, far right) and see which is most open.
        # the depth map uses black for close (0) and white for far away(255), so we invert that
        # to get an estimate of distance.
        bands = np.hsplit(top, [50,100,150,200]);
        mins = [np.min(x) for x in bands]
        max = np.argmax(mins)    
        distance = mins[max]

        # sanity check on what is directly in front of us (slot 2 in our hsplit)
        current = mins[2]
        print ("max distance=", current)
        print ("distance=", current)
#         if (current <20):
#             client.hoverAsync().join()
#             airsim.wait_key("whoops - we are about to crash, so stopping!")
    
        pitch, roll, yaw  = airsim.to_eularian_angles(client.simGetVehiclePose().orientation)

        if (distance > current + 10):
        
            # we have a 90 degree field of view (pi/2), we've sliced that into 5 chunks, each chunk then represents
            # an angular delta of the following pi/10.
            change = 0
            driving = max
            if (max == 0):
                change = -2 * pi / 10
            elif (max == 1):
                change = -pi / 10
            elif (max == 2):
                change = 0 # center strip, go straight
            elif (max == 3):
                change = pi / 10
            else:
                change = 2*pi/10
    
            yaw = (yaw + change)
            vx = turn_speed*math.cos(yaw);
            vy = turn_speed*math.sin(yaw);
            print ("switching angle", math.degrees(yaw), vx, vy, min, distance, current)
    
        if (vx == 0 and vy == 0):
            vx = math.cos(yaw);
            vy = math.sin(yaw);

        #print ("distance=", current)
        client.moveByVelocityZAsync(fly_speed*vx, fly_speed*vy,-25, 1, airsim.DrivetrainType.ForwardOnly, airsim.YawMode(False, 0)).join()

        x = int(driving * 50)
        cv2.rectangle(gray, (x,0), (x+50,50), (0,255,0), 2)
        cv2.imshow("Top", top)

    key = cv2.waitKey(1) & 0xFF;
    if (key == 27 or key == ord('q') or key == ord('x')):
        break;
        


max distance= 107
distance= 107
switching angle 215.99995903307672 -0.40450870732285976 -0.2938923369194729 <built-in function min> 127 107
max distance= 107
distance= 107
max distance= 185
distance= 185
switching angle -107.5013546854304 -0.15036417441890074 -0.47685492033879906 <built-in function min> 209 185
max distance= 200
distance= 200
max distance= 193
distance= 193
max distance= 193
distance= 193
max distance= 185
distance= 185
max distance= 178
distance= 178
max distance= 178
distance= 178
max distance= 164
distance= 164
max distance= 144
distance= 144
switching angle -71.5011122267247 0.1586431237345445 -0.47416490727567134 <built-in function min> 170 144
max distance= 151
distance= 151
switching angle -91.07138983485274 -0.009349095234157 -0.49991258677723116 <built-in function min> 170 151
max distance= 164
distance= 164
max distance= 151
distance= 151
max distance= 127
distance= 127
max distance= 144
distance= 144
switching angle -109.07081585041158 -0.16336826892344977 -

KeyboardInterrupt: 

In [ ]:
bands

In [ ]:
rawImage = np.fromstring(result, np.int8)